## Face Recognition Based Attendance System

In [5]:
pip install opencv-python --upgrade

Note: you may need to restart the kernel to use updated packages.


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/fb/c4/f574ba6f04e6d7bf8c38d23e7a52389566dd7631fee0bcdd79ea07ef2dbf/opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB 660.6 kB/s eta 0:00:58
   ---------------------------------------- 0.1/38.1 MB 656.4 kB/s eta 0:00:58
   ---------------------------------------- 0.1/38.1 MB 950.9 kB/s eta 0:00:40
   ---------------------------------------- 0.2/38.1 MB 1.1 MB/s eta 0:00:34
   ---------------------------------------- 0.4/38.1 MB 1.5 MB/s eta 0:00:25
    --------------------------------------- 0.5/38.1 MB 1.6 MB/s eta 0:00:24
    --------------------------------------- 0.6/38.1 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.6/38.1 MB 1.8 MB/s eta 0:00:22
    --------------------------------------- 0.7/38.1 MB 1.7 MB/s eta 0:00:22
    ------

In [9]:
import cv2
print(cv2.__version__)

4.8.0


In [4]:
# write the name just once
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import pickle
import pandas as pd  # Import pandas library for handling Excel files

path = 'C:/Users/nabee/Desktop/cvproject/student_images'
images = []
classNames = []
mylist = os.listdir(path)

for cl in mylist:
    try:
        curImg = cv2.imread(f'{path}/{cl}')
        if curImg is not None:
            images.append(curImg)
            classNames.append(os.path.splitext(cl)[0])
        else:
            print(f"Error loading image: {cl}")
    except Exception as e:
        print(f"Error processing image: {cl}. Error: {str(e)}")

def findEncodings(images):
    encodeList = []
    confidences = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)
        if len(encoded_face) > 0:
            encodeList.append(encoded_face[0])
            # Calculate the confidence score and store it
            face_distances = face_recognition.face_distance(encoded_face_train, encoded_face[0])
            min_distance = min(face_distances)
            confidence = 1 - min_distance
            confidences.append(confidence)
        else:
            print("No face found in the image.")
            confidences.append(0)  # If no face found, set confidence to 0
    return encodeList, confidences

encoded_face_train, confidences = findEncodings(images)

# Define a threshold for confidence
confidence_threshold = 0.6  # Adjust this value as needed

# Set to store unique attendance names
attendance_names_set = set()

def markAttendance(name, confidence, file_path):
    # Check if the file exists, and if not, create it with a header
    if not os.path.isfile(file_path):
        with open(file_path, 'w') as f:
            f.write('Name,Confidence,Time,Date\n')

    if name not in attendance_names_set:
        with open(file_path, 'a') as f:
            now = datetime.now()
            time = now.strftime('%I:%M:%S:%p')
            date = now.strftime('%d-%B-%Y')
            entry = f'{name},{confidence},{time},{date}\n'
            f.write(entry)
            attendance_names_set.add(name)  # Add the name to the set

# Create a new Excel file with a unique name based on the current timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
attendance_file_path = f'Attendance_{timestamp}.csv'

cap = cv2.VideoCapture(0)  # 0 represents the default camera (usually the built-in webcam)

while True:
    success, img = cap.read()

    if not success:
        continue  # Skip empty frames

    # Resize the frame
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    faces_in_frame = face_recognition.face_locations(imgS)
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

    for encode_face, faceloc, confidence in zip(encoded_faces, faces_in_frame, confidences):
        if confidence >= confidence_threshold:
            matches = face_recognition.compare_faces(encoded_face_train, encode_face)
            faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
            matchIndex = np.argmin(faceDist)
            print(matchIndex)
            if matches[matchIndex]:
                name = classNames[matchIndex].upper().lower()
                y1, x2, y2, x1 = faceloc
                # Since we scaled down by 4 times
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                markAttendance(name, confidence, attendance_file_path)

    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the attendance data to the Excel file and close thqe file
print(f"Attendance saved to {attendance_file_path}")
cap.release()
cv2.destroyAllWindows()

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
3
3
3
3
3
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
3
0
0
0
3
3
3
3
Attendance saved to Attendance_2023-09-21_17-00-59.csv


## Face Recognition Based Attendance System with confidence Function

In [13]:
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime

path = 'C:/Users/nabee/Desktop/cvproject/student_images'
images = []
classNames = []
mylist = os.listdir(path)

for cl in mylist:
    try:
        curImg = cv2.imread(f'{path}/{cl}')
        if curImg is not None:
            images.append(curImg)
            classNames.append(os.path.splitext(cl)[0])
        else:
            print(f"Error loading image: {cl}")
    except Exception as e:
        print(f"Error processing image: {cl}. Error: {str(e)}")

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)
        if len(encoded_face) > 0:
            encodeList.append(encoded_face[0])
        else:
            print("No face found in the image.")
    return encodeList

encoded_face_train = findEncodings(images)

# Set to store unique attendance names
attendance_names_set = set()
# Minimum confidence threshold
min_confidence = 0.50

def markAttendance(name, file_path, confidence):
    if confidence >= min_confidence:  # Check if confidence meets the threshold
        # Check if the file exists, and if not, create it with a header
        if not os.path.isfile(file_path):
            with open(file_path, 'w') as f:
                f.write('Name,Time,Date,Confidence\n')

        if name not in attendance_names_set:
            with open(file_path, 'a') as f:
                now = datetime.now()
                time = now.strftime('%I:%M:%S:%p')
                date = now.strftime('%d-%B-%Y')
                entry = f'{name},{time},{date},{confidence}\n'
                f.write(entry)
                attendance_names_set.add(name)  # Add the name to the set


# Create a new Excel file with a unique name based on the current timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
attendance_file_path = f'Attendance_{timestamp}.csv'

# Take pictures from the webcam
cap  = cv2.VideoCapture(0)
while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    # Get face locations
    faces_in_frame = face_recognition.face_locations(imgS)
    
    # Encode detected faces
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
    
    for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
        matchIndex = np.argmin(faceDist)
        confidence = 1 - faceDist[matchIndex]  # Calculate confidence score

        if matches[matchIndex] and confidence >= min_confidence:
            name = classNames[matchIndex].upper().lower()
            y1, x2, y2, x1 = faceloc
            # Since we scaled down by 4 times
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, f'{name} ({confidence:.2f})', (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
            markAttendance(name, attendance_file_path, confidence)

    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the attendance data to the Excel file and close the file
print(f"Attendance saved to {attendance_file_path}")
cap.release()
cv2.destroyAllWindows()

Attendance saved to Attendance_2023-09-21_17-13-34.csv
